# Working with the Polar Data   

In [111]:
import scipy.signal as sig
import scipy.stats as stats

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.animation as animation
import matplotlib.colors as colors
%matplotlib

import os
import pandas as pd 
import numpy as np
from datetime import datetime
import xarray as xr
from mpl_toolkits.mplot3d import Axes3D
import time


Using matplotlib backend: Qt5Agg


### heave_spectrum for 2016-12-29

In [135]:
fn_mkiv_heave_spectrum = r"mkiv.h5"

h5_heave_spectrum = pd.HDFStore(fn_mkiv_heave_spectrum)

heave_spectrum = h5_heave_spectrum.select('/heave_spectrum')

df =heave_spectrum[(heave_spectrum.index >= "2016-12-29") & (heave_spectrum.index < "2016-12-30")].iloc[0]


del df[0]
del df[1]
del df[2]
del df['file_name']

df.plot()

3      0.000179939
4       0.00155261
5         0.024902
6         0.190556
7          1.13564
8          1.28043
9         0.523192
10         3.36085
11         10.9374
12         11.9078
13         7.59274
14         6.43784
15         6.70057
16         5.62484
17         5.65304
18         5.11508
19         4.33704
20         4.67483
21         4.74548
22         4.53667
23         2.68369
24         2.00811
25         2.23043
26         2.39215
27         1.50259
28         1.68572
29         1.81701
30         1.57175
31         1.51013
32         1.38015
          ...     
73       0.0316566
74       0.0271112
75       0.0358717
76       0.0287877
77       0.0217573
78       0.0136665
79       0.0275209
80       0.0189148
81        0.024902
82       0.0311853
83       0.0147309
84        0.020388
85       0.0174606
86       0.0114724
87       0.0129351
88        0.012679
89      0.00688914
90       0.0136665
91       0.0102261
92      0.00490348
93      0.00335329
94      0.00

## Grabbing the Polar h5 and converting it into a Panda's data frame. 
- I will be working with 64 rotation  

In [108]:
fn_polar_data = r"radar_rot_5.h5"

second_64_rotation_pd = pd.read_hdf(fn_polar_data,key='/polar_image', start=262144, stop=524288)

Making a new H5 file with 64 rotations

In [ ]:
#second_64_rotation_pd.to_hdf('rotation64.h5','polar_image', format='table', complevel=9, complib='blosc')

### Using numpy to split the polar data into 64 dataframes

In [112]:
list_with_64_rotation_df = np.split(second_64_rotation_pd, 64)

Alternative method for the above np split

In [20]:
#rot_dict = {}
#for index, x in enumerate(list_rot_df):
#    rot_dict[index] = x


### Xarray
- Using xArray to concat the 64 data frame into one and selecting a subset distance and the angle (256*256) of the polar data 
- These functions are used to create a DataSet that contains a DataFrame with 64 rotation data,
  and each rotation will contain 256*256 Angle and Distance_Cells data


In [113]:
def rotational_and_angular_datetime_list(dataframe_list):
    #This function's parameter is a numpy array with 64 dataframes
    
    # This will contain a list of rotational DateTime for each rotation 
    rotational_datatime = [] # (64 rotation)
    # This will contain a list of angular DateTime for each beam in a rotation 
    angular_datetime = [] # (256 beam per rotation)

    # This will hold a list of 64 rotations
    # that starts from 3269 and ends brfore 3525 index number for each rotation 
    rotational_df_with_256_rows = []
    
    # this loops 64 times 
    for x in range(len(dataframe_list)):

        # This looks for index that starts from 3269 and ends before 3525
        # and appends it to rotational_df_with_256_rows list
        rotational_df_with_256_rows.append(dataframe_list[x].iloc[3269:3525])
         
        # This will grab the angular datetime column from each rotation 
        # and append it to the angular_datetime list 
        angular_datetime.append(rotational_df_with_256_rows[x].index.values)
        
        
        
        # This will grab the first datetime from each angular_datetime list for the 
        # rotational_datatime list. 
        rotational_datatime.append(angular_datetime[x][0])
        
    """print(angular_datetime[0])
    print("=========================")
    print(rotational_datatime)"""
    
    rotational_and_angular_datetime = {
        'rotational_datatime' : rotational_datatime, 
        'angular_datetime' : angular_datetime 
    }
    
    return rotational_and_angular_datetime

In [114]:
rotational_and_angular_datetime =rotational_and_angular_datetime_list(list_with_64_rotation_df)

### UTM 29 North values for Mkiv
- Extracting "mkiv_opp_adj.h5" file containing the UTM29N values for the mkiv. 
- Using scatter plot on the UTM29N values 

In [115]:
fn_mkiv_opp_adj = r"mkiv_opp_adj.h5"

h5 = pd.HDFStore(fn_mkiv_opp_adj)

h5.keys()

mkiv_gps_df = h5.select('/mkiv_opp_adj')

mkiv_x_opp_df = mkiv_gps_df['mkiv_x_opp_df'].as_matrix()
mkiv_y_adj_df = mkiv_gps_df['mkiv_y_adj_df'].as_matrix()

plt.figure("fig2")
plt.scatter(mkiv_gps_df['mkiv_x_opp_df'], mkiv_gps_df['mkiv_y_adj_df'])
plt.show()

### Creating a DataSet

In [118]:
def create_a_xr_dataarray(rotation_list, rotational_and_angular_datetime):
    
    # The angular_index_range and distance_cells_index_range
    # will be the coordinates for the angular_cells and distance_cells dimensions
    angular_index_range = list(range(4096)) 
    distance_cells_index_range = list(range(1024))
    
    # This list will hold 64 DataArrays with angular_cells and distance_cells dimension
    # and angular_index_range and distance_cells_index_range as the coordinates
    list_x_dataarray = []
    
    # iterating through each rotations to create a dataArray
    # and appending it to the list_x_dataarray.
    # reset_index function is used to create a new integer index coulmn and
    # drop=True parameter will delete the previous index in the dataFrame.

    for index, x in enumerate(rotation_list[0:]):
        list_x_dataarray.append(xr.DataArray(x.reset_index(drop=True),
                                             coords=[angular_index_range, distance_cells_index_range], 
                                             dims=['angular_cells', 'distance_cells']))
    
    # the xArray concat function is to create an ND DataArray using the list of 64 DataArray (list_x_dataarray).
    # An additional dimension is attached to the DataArray that represents each rotation.
    xr_dataframe_rot = xr.concat(list_x_dataarray, dim='rotational_datatime' ) #rot_index_timeStamp  coords= list(range(64)),
    
    boy_locations_polar_data = select_a_subset_of_the_dataarray(xr_dataframe_rot)
    
    return create_a_xr_dataset(boy_locations_polar_data, rotational_and_angular_datetime )



def select_a_subset_of_the_dataarray(rotation_dataarray):
    """
    This function selectes a subset of angular 
    and distance cells from each rader rotations (64).
    The subset holds backscatter data for the MKIV's location. 
    
    This function's parameter contains an ND DataArray 
    (64 rotation, 4096 angular_cells, 1024 distance_cells)
    
    and returns an ND DataArray with 
    (64 rotation, 256 angular_cells, 256 distance_cells)
    
    """
    
    rotational_datatime_range = list(range(0,64)) 
    angular_range = list(range(3269,3525)) 
    distance_cells_range = list(range(215,471))
    
    boy_locations_polar_data = rotation_dataarray[rotational_datatime_range, 
                                                  angular_range, 
                                                  distance_cells_range]
    return boy_locations_polar_data
    

def create_a_xr_dataset(dataarray, datatime_dictionary):
    """
    This function takes an ND DataArray with
    (64 rotation, 256 angular_cells, 256 distance_cells)
    
    creates coordinate for the 64 rotation(rotational_datatime) 
    by using the rotational_datatime list defined above and 
    for each 256 rows for the 64 rotations using the angular_datetime list. 
    
    The Function returns a DataSet containing a DataArray variable 
    
    """
    ds = xr.Dataset({'second_rotation_64': dataarray},
                coords = {
                    'rotational_datatime': datatime_dictionary['rotational_datatime'],
                    'angular_datetime': (
                        ['rotational_datatime', 'angular_cells'], datatime_dictionary['angular_datetime']
                    )
                    ,
                    'UTM29N_lon' : (
                        ['angular_cells', 'distance_cells'], mkiv_y_adj_df
                    ),
                    'UTM29N_lat' : (
                        ['angular_cells', 'distance_cells'], mkiv_x_opp_df
                    )
                })
    return ds
    

In [119]:
ds = create_a_xr_dataarray(list_with_64_rotation_df, rotational_and_angular_datetime)

In [120]:
ds.second_rotation_64[1]

<xarray.DataArray 'second_rotation_64' (angular_cells: 256, distance_cells: 256)>
array([[ 8183.,  8945.,  9252., ...,  8131.,  8728.,  8892.],
       [ 8396.,  8979.,  9203., ...,  7832.,  8618.,  8927.],
       [ 8610.,  9012.,  9155., ...,  7533.,  8508.,  8962.],
       ..., 
       [ 9069.,  9315.,  9386., ...,  8567.,  8421.,  9283.],
       [ 9375.,  9454.,  9395., ...,  7907.,  8101.,  9188.],
       [ 9464.,  9522.,  9437., ...,  7560.,  7829.,  8947.]], dtype=float32)
Coordinates:
  * angular_cells        (angular_cells) int32 3269 3270 3271 3272 3273 3274 ...
  * distance_cells       (distance_cells) int32 215 216 217 218 219 220 221 ...
    rotational_datatime  datetime64[ns] 2016-12-29T00:08:34.339476
    angular_datetime     (angular_cells) datetime64[ns] 2016-12-29T00:08:34.339476 ...
    UTM29N_lon           (angular_cells, distance_cells) float64 6.598e+05 ...
    UTM29N_lat           (angular_cells, distance_cells) float64 6.489e+06 ...

In [28]:
print("First rotation angular_datetime\nNumber of Rows :", len(ds.first_rotation_64[0].angular_datetime), "\nFirst row :", ds.first_rotation_64[0].angular_datetime[0].values)

First rotation angular_datetime
Number of Rows : 256 
First row : 2016-12-29T00:08:33.018080000


In [122]:
fig = plt.figure(figsize=(104, 104), dpi=65, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_surface(ds.UTM29N_lat.values, ds.UTM29N_lon.values, ds.second_rotation_64[0].values,cmap=cm.Blues,linewidth=0, antialiased=False)
#lt.cla()
#surf = ax.plot_surface(ds.UTM29N_lat.values, ds.UTM29N_lon.values, ds.second_rotation_64[1].values,cmap=cm.Blues, linewidth=0, antialiased=False)

ax.set_aspect(0.31)
# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=10)

ax.set_xlabel('X UTM29N_lat Label')
ax.set_ylabel('Y UTM29N_lon Label')
ax.set_zlabel('Z backscatter Label')

plt.show()

### making contourf animation 
- using the UTM29N values as X and Y 
- and 64 rotation backscatter values as the Z 

In [126]:
X = ds.UTM29N_lat.values
Y = ds.UTM29N_lon.values
Z = ds.second_rotation_64

In [127]:
def update_graph(num):
    plt.cla()
    line = ax.contourf(X, Y, Z[num].values, 64, cmap=cm.Blues,linewidth=0, antialiased=False,corner_mask=True )
    title = ax.set_title('Rotation')
    title.set_text('Rotation={}'.format(num))
    


fig = plt.figure(figsize=(16, 12), dpi=80, facecolor='w', edgecolor='k')

ax = fig.add_subplot(111)
title = ax.set_title('Rotation')


line = ax.contourf(X, Y, Z[0].values, 64, cmap=cm.Blues,linewidth=0, antialiased=False,corner_mask=True)



fig.colorbar(line, ax=ax, extend='max', aspect=10)

ax.set_xlabel('X UTM29N_lat Label')
ax.set_ylabel('Y UTM29N_lon Label')


ani = matplotlib.animation.FuncAnimation(fig, update_graph, 64, 
                               interval=35, blit=False)
plt.show()

### making plot_surface animation
- using the UTM29N values as X and Y
- and 64 rotation backscatter values as the Z

In [128]:
def update_graph(num):
    plt.cla()
    line = ax.plot_surface(X, Y, Z[num].values,  cmap=cm.Blues,linewidth=0, antialiased=False )
    title = ax.set_title('Rotation')
    title.set_text('Rotation={}'.format(num))
    ax.set_aspect(0.35)


fig = plt.figure(figsize=(80, 80), dpi=65, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111, projection='3d')
title = ax.set_title('Rotation')


line = ax.plot_surface(X, Y, Z[0].values, cmap=cm.Blues,linewidth=0, antialiased=False)
ax.set_aspect(0.35)


fig.colorbar(line, ax=ax, extend='max', aspect=10)
#fig.colorbar(line, shrink=0.5, aspect=10)
ax.set_xlabel('X UTM29N_lat Label')
ax.set_ylabel('Y UTM29N_lon Label')
ax.set_zlabel('Z backscatter Label')
#ax.view_init(90, -90)
ax.view_init(80, -90)

ani = matplotlib.animation.FuncAnimation(fig, update_graph, 64, 
                               interval=35, blit=False)
plt.show()

### Working with the welch algorithm

In [145]:
def create_frequency_welch_df(backscatters_for_point_over_time, frequency_of_samples=1/1.3):
    return create_welch_df(backscatters_for_point_over_time, frequency_of_samples)

def create_wavelength_welch_df(backscatters_returns_from_one_pulse, distance_between_cells=2.998): #5.995
    return create_welch_df(backscatters_returns_from_one_pulse, distance_between_cells)

def create_welch_df(dataframe, spacing):
    """Takes a single column spacing is either in metres or seconds"""
    x,y = sig.welch(dataframe, spacing, noverlap=None)
    dic = {'x':x,'y':y}
    welch_df = pd.DataFrame(dic)
    return welch_df

def calc_welch(welch_df,fig_name, psd_dic = {}, save = False, line_name = ''):
    """ Given dataframe will run welch's methods on the values, printing out the maxmimum power spectral density and the 
        frequency it occurs at, plots the output and returns the data
    """
    psd_dic[line_name] = [welch_df.x.max(), welch_df.y.loc[welch_df.y==welch_df.y.max()]] 

    
    plt.figure(fig_name)
    plt.plot(welch_df.x,welch_df.y, label=line_name)
    plt.ylabel('S(f) m^2/Hz')
    plt.xlabel('Frequency (Hz)')
    #plt.legend()
    return psd_dic

In [147]:
mkiv_location_per_rotation = []
for index, x in enumerate(ds.second_rotation_64):
    mkiv_location_per_rotation.append(x.sel(angular_cells=3433,distance_cells=335).values)
    
    # Alternative methods
    ## x.loc[3433,335]
    ## x.loc[dict(angular_cells=3433,distance_cells=335)]
    
df  = pd.DataFrame(data=mkiv_location_per_rotation).T

# Using the Welch algorithm to convert the wave form time domain to the frequency domain
welch_df = create_frequency_welch_df(df.iloc[0])
wavelength_df = calc_welch(welch_df, "number4")  

C:\Users\le12jm\AppData\Local\Continuum\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1637: UserWarning: nperseg = 256 is greater than input length  = 64, using nperseg = 64
  .format(nperseg, input_length))
